# Test Merging

In [1]:
import pandas as pd
import os
from glob import glob
from tqdm import tqdm
from IPython.display import display, HTML, Markdown

# 1) Load Market Data

## Why not?

In [2]:
def get_ticker_name(path):
    """
    get ticker name from path

    :param path: path to market df
    :type path: str
    :return: ticker name
    :rtype: str
    """
    name = path.split("/")[-1].split(".")[0]
    name = name.replace(" ", "_") + "_return"
    return name 



def get_market_df(path):
    """
    Get ticker dataframe from path.
    
    We always drop the first two rows
    with the information
    
    'field, DAY_TO_DAY_TOT_RETURN_GROSS_DVDS
     date,'
     
    The total return is divided by 100.
    The return column named as
    "get_ticker_name(path)".

    :param path: path to market df
    :type path: str
    :return: market dataframe
    :rtype: pd.DataFrame
    """
    target_name = get_ticker_name(path)
    market = pd.read_csv(path)
    market = market.drop([0, 1], 0) # drop first 2 lines
    market.columns = ["date", target_name]
    market.loc[:, "date"] = pd.to_datetime(market.date)
    market.loc[:, target_name] = market[target_name].astype("float") / 100
    return market.reset_index(drop=True)

## Example

In [3]:
paths = glob("data/crsp/nyse/*.csv")
paths.sort()
path = paths[0]

df1 = get_market_df(path)
df1.head(3)

,date,0047042D_US_Equity_return
0,2000-01-03,-0.015385
1,2000-01-04,0.000000
2,2000-01-05,-0.015625


## Daniel func

In [4]:
def adj_find_target_str_bbg_data(df, target_str='date'):
    if df.columns[0] != target_str:
        df.columns = df.iloc[0]
        while df.columns[0] != target_str:
            df.drop(df.index[0], inplace=True)
            df.columns = df.iloc[0]
        df.drop(df.index[0], inplace=True)
    return df

## Example

In [5]:
df2 = adj_find_target_str_bbg_data(pd.read_csv(path))
df2 = df2.reset_index(drop=True)
df2.columns.name = ""
df2.head(3)

,date,NaN
0,2000-01-03,-1.5385
1,2000-01-04,0.0
2,2000-01-05,-1.5625


## test 1) Nyse

In [6]:
paths = glob("data/crsp/nyse/*.csv")

for p in tqdm(paths):
    df1 = get_market_df(path)
    df2 = adj_find_target_str_bbg_data(pd.read_csv(path))
    assert df1.shape == df2.shape

100%|██████████| 2163/2163 [00:16<00:00, 133.90it/s]


## test 2) Nasdaq

In [7]:
paths = glob("data/crsp/nasdaq/*.csv")

for p in tqdm(paths):
    df1 = get_market_df(path)
    df2 = adj_find_target_str_bbg_data(pd.read_csv(path))
    assert df1.shape == df2.shape

100%|██████████| 10185/10185 [01:28<00:00, 115.40it/s]


In [8]:
df2 = df2.reset_index(drop=True)
df2.columns.name = ""
df2.head(3)

,date,NaN
0,2000-01-03,-1.5385
1,2000-01-04,0.0
2,2000-01-05,-1.5625


# 2) Merge function


In [9]:
def merge_data(df_list, freq='D'):
    list_out = []
    for i, df in enumerate(df_list):
        # print(i)
        df_loop = df.resample(freq).mean()
        list_out.append(df_loop)
    df_out = pd.concat(list_out, axis=1)

    return df_out


In [10]:
paths = glob("data/crsp/nyse/*.csv")
paths.sort()

df1 =  get_market_df(paths[1])
df2 = get_market_df(paths[2])
df1, df2 = df1.set_index("date"),df2.set_index("date")
display(Markdown("## DF1 "))
display(HTML(df1.head(6).to_html()))
display(Markdown("#### DF1 shape = {}".format(df1.shape)))

display(Markdown("## DF2 "))
display(HTML(df2.head(6).to_html()))
display(Markdown("#### DF2 shape = {}".format(df2.shape)))


## DF1 

,0062761Q_US_Equity_return
date,
2000-01-03,0.000000
2000-01-04,-0.007692
2000-01-05,0.007752
2000-01-06,0.000000
2000-01-07,-0.007692
2000-01-10,0.007752


#### DF1 shape = (1723, 1)

## DF2 

,0111145D_US_Equity_return
date,
2000-01-03,-0.030769
2000-01-04,0.005952
2000-01-05,0.027613
2000-01-06,0.015355
2000-01-07,0.022684
2000-01-10,-0.001848


#### DF2 shape = (3005, 1)

## Why not?

In [11]:
def merge_data(df_list, freq='D'):
    """
    Merge all df's in the list 'df_list'.
    We assume that all df's are indexed
    by date.
    
    We resample all df's using the frequency
    "freq", and we concatenate them into 
    a single dataframe.

    :param path: list of dataframes
    :type path: [pd.DataFrame]
    :return: merged dataframe
    :rtype: pd.DataFrame
    """
    list_out = []
    for df in df_list:
        df_loop = df.resample(freq).mean()
        list_out.append(df_loop)

    return pd.concat(list_out, axis=1)


## Merge between tickers

In [12]:
merged1 =  merge_data([df1, df2])

display(Markdown("## merged1 "))
display(HTML(merged1.head(7).to_html()))
display(Markdown("#### merged1 shape = {}".format(merged1.shape)))

## merged1 

,0062761Q_US_Equity_return,0111145D_US_Equity_return
date,,
2000-01-03,0.000000,-0.030769
2000-01-04,-0.007692,0.005952
2000-01-05,0.007752,0.027613
2000-01-06,0.000000,0.015355
2000-01-07,-0.007692,0.022684
2000-01-08,NaN,NaN
2000-01-09,NaN,NaN


#### merged1 shape = (4359, 2)

## Merge between tickers and gtrend

In [13]:
gtrends = pd.read_csv("data/gtrends.csv")
gtrends.loc[:, "date"] = pd.to_datetime(gtrends.date)
gtrends = gtrends.set_index("date")

merged2 =  merge_data([df1, gtrends])

display(Markdown("## merged2 "))
display(HTML(merged2.head(7).to_html()))
display(Markdown("#### merged2 shape = {}".format(merged2.shape)))

merged3 = merged2.dropna()

display(Markdown("## merged3 =merged2.dropna() "))
display(Markdown("#### merged3 shape = {}".format(merged3.shape)))
display(HTML(merged3.head(2).to_html()))
display(HTML(merged3.tail(2).to_html()))
display(HTML(df1.tail(2).to_html()))

## merged2 

,0062761Q_US_Equity_return,BUY AND HOLD,DOW JONES,act,arts,bank,banking,blacklist,bonds,bubble,business,buy,cancer,car,carolina,case,cash,ceo,chance,college,color,committee,community,companies,conflict,consume,consumption,corporation,council,county,court,crash,credit,crisis,culture,debt,default,democratic,derivatives,development,district,dividend,dow jones,earnings,economic,economics,economy,elected,election,elections,energy,environment,fed,federal,finance,financial,financial markets,fine,firm,fond,food,forex,founded,freedom,fun,gain,gains,garden,georgia,global,gold,government,governor,greed,growth,happy,headlines,health,hedge,holiday,home,house,housing,illinois,inc,industry,inflation,invest,investment,judge,justice,kentucky,kitchen,labor,law,legal,leverage,lifestyle,loss,ltd,management,market,marketing,markets,marriage,massachusetts,media,members,metals,million,minister,ministry,missouri,money,movement,movie,nasdaq,nyse,office,ohio,oil,opportunity,ore,party,pennsylvania,police,political,politics,portfolio,present,president,products,profit,project,rare earths,religion,representatives,republican,restaurant,return,returns,revenue,rich,rights,ring,risk,seats,secretary,security,sell,senate,served,service,services,short sell,short selling,social,society,stats,stock market,stocks,success,technology,tennessee,texas,tourism,trader,train,transaction,travel,unemployment,union,vermont,virginia,voters,votes,war,washington,water,william,wisconsin,world,york
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000-01-03,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,-0.007692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,0.007752,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

#### merged2 shape = (7580, 183)

## merged3 =merged2.dropna() 

#### merged3 shape = (721, 183)

,0062761Q_US_Equity_return,BUY AND HOLD,DOW JONES,act,arts,bank,banking,blacklist,bonds,bubble,business,buy,cancer,car,carolina,case,cash,ceo,chance,college,color,committee,community,companies,conflict,consume,consumption,corporation,council,county,court,crash,credit,crisis,culture,debt,default,democratic,derivatives,development,district,dividend,dow jones,earnings,economic,economics,economy,elected,election,elections,energy,environment,fed,federal,finance,financial,financial markets,fine,firm,fond,food,forex,founded,freedom,fun,gain,gains,garden,georgia,global,gold,government,governor,greed,growth,happy,headlines,health,hedge,holiday,home,house,housing,illinois,inc,industry,inflation,invest,investment,judge,justice,kentucky,kitchen,labor,law,legal,leverage,lifestyle,loss,ltd,management,market,marketing,markets,marriage,massachusetts,media,members,metals,million,minister,ministry,missouri,money,movement,movie,nasdaq,nyse,office,ohio,oil,opportunity,ore,party,pennsylvania,police,political,politics,portfolio,present,president,products,profit,project,rare earths,religion,representatives,republican,restaurant,return,returns,revenue,rich,rights,ring,risk,seats,secretary,security,sell,senate,served,service,services,short sell,short selling,social,society,stats,stock market,stocks,success,technology,tennessee,texas,tourism,trader,train,transaction,travel,unemployment,union,vermont,virginia,voters,votes,war,washington,water,william,wisconsin,world,york
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004-01-02,0.001867,0.0,1.0,3.0,0.0,38.0,-1.0,1.0,-4.0,-2.0,20.0,-10.0,1.0,0.0,-6.0,-5.0,-1.0,1.0,-1.0,18.0,-2.0,-1.0,14.0,-3.0,1.0,0.0,0.0,6.0,2.0,30.0,13.0,0.0,14.0,0.0,2.0,0.0,1.0,0.0,0.0,7.0,10.0,0.0,2.0,0.0,1.0,-3.0,-1.0,0.0,-2.0,1.0,4.0,0.0,0.0,14.0,5.0,15.0,1.0,-2.0,4.0,1.0,-18.0,0.0,0.0,-6.0,-7.0,-2.0,2.0,-3.0,5.0,5.0,2.0,9.0,1.0,1.0,-1.0,-7.0,1.0,24.0,1.0,-6.0,1.0,10.0,2.0,16.0,7.0,6.0,1.0,0.0,3.0,-1.0,0.0,-1.0,-6.0,3.0,17.0,0.0,1.0,-1.0,1.0,3.0,11.0,1.0,4.0,0.0,5.0,0.0,-4.0,-1.0,1.0,-2.0,2.0,0.0,-2.0,2.0,2.0,-5.0,2.0,1.0,15.0,5.0,-2.0,0.0,0.0,-2.0,4.0,4.0,4.0,0.0,-4.0,0.0,3.0,10.0,0.0,3.0,0.0,-2.0,-1.0,-1.0,-13.0,-5.0,-2.0,5.0,1.0,3.0,-7.0,1.0,0.0,7.0,12.0,2.0,2.0,0.0,17.0,27.0,-1.0,1.0,5.0,2.0,-4.0,1.0,2.0,1.0,6.0,-2.0,11.0,-1.0,-5.0,-2.0,0.0,-4.0,2.0,10.0,-8.0,8.0,0.0,0.0,5.0,20.0,2.0,4.0,0.0,14.0,11.0
2004-01-05,0.003728,-1.0,0.0,3.0,3.0,-7.0,-3.0,0.0,-1.0,-3.0,1.0,-10.0,6.0,-2.0,-4.0,0.0,0.0,3.0,-1.0,32.0,7.0,1.0,38.0,1.0,-2.0,-1.0,2.0,-3.0,1.0,4.0,1.0,4.0,2.0,-1.0,2.0,1.0,-2.0,1.0,-2.0,0.0,27.0,0.0,1.0,2.0,1.0,2.0,-1.0,1.0,1.0,1.0,-7.0,2.0,-1.0,-4.0,0.0,1.0,2.0,-7.0,4.0,0.0,-4.0,-1.0,2.0,0.0,5.0,1.0,-1.0,-3.0,-8.0,-2.0,3.0,4.0,3.0,0.0,-2.0,-4.0,0.0,-1.0,1.0,0.0,-13.0,-7.0,0.0,-7.0,2.0,-2.0,3.0,0.0,2.0,2.0,-3.0,0.0,-4.0,-2.0,6.0,5.0,-1.0,0.0,1.0,-1.0,10.0,-3.0,4.0,0.0,15.0,-2.0,0.0,-1.0,-2.0,-1.0,0.0,3.0,0.0,0.0,3.0,-10.0,1.0,1.0,-8.0,-11.0,-1.0,1.0,1.0,-11.0,-5.0,-2.0,-3.0,3.0,1.0,-1.0,3.0,2.0,0.0,-3.0,0.0,0.0,0.0,1.0,-8.0,-2.0,1.0,-2.0,1.0,-2.0,2.0,-4.0,-2.0,-1.0,5.0,-4.0,0.0,1.0,-9.0,6.0,0.0,0.0,-4.0,1.0,-2.0,3.0,0.0,0.0,-3.0,-2.0,-4.0,4.0,4.0,-8.0,2.0,3.0,-6.0,-6.0,-4.0,1.0,1.0,1.0,4.0,-1.0,3.0,5.0,1.0,4.0,-15.0


,0062761Q_US_Equity_return,BUY AND HOLD,DOW JONES,act,arts,bank,banking,blacklist,bonds,bubble,business,buy,cancer,car,carolina,case,cash,ceo,chance,college,color,committee,community,companies,conflict,consume,consumption,corporation,council,county,court,crash,credit,crisis,culture,debt,default,democratic,derivatives,development,district,dividend,dow jones,earnings,economic,economics,economy,elected,election,elections,energy,environment,fed,federal,finance,financial,financial markets,fine,firm,fond,food,forex,founded,freedom,fun,gain,gains,garden,georgia,global,gold,government,governor,greed,growth,happy,headlines,health,hedge,holiday,home,house,housing,illinois,inc,industry,inflation,invest,investment,judge,justice,kentucky,kitchen,labor,law,legal,leverage,lifestyle,loss,ltd,management,market,marketing,markets,marriage,massachusetts,media,members,metals,million,minister,ministry,missouri,money,movement,movie,nasdaq,nyse,office,ohio,oil,opportunity,ore,party,pennsylvania,police,political,politics,portfolio,present,president,products,profit,project,rare earths,religion,representatives,republican,restaurant,return,returns,revenue,rich,rights,ring,risk,seats,secretary,security,sell,senate,served,service,services,short sell,short selling,social,society,stats,stock market,stocks,success,technology,tennessee,texas,tourism,trader,train,transaction,travel,unemployment,union,vermont,virginia,voters,votes,war,washington,water,william,wisconsin,world,york
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2006-11-08,0.003296,0.0,1.0,-1.0,-2.0,-2.0,0.0,0.0,0.0,0.0,-6.0,-4.0,0.0,-2.0,0.0,-2.0,-1.0,0.0,0.0,-3.0,-2.0,1.0,-4.0,-1.0,0.0,0.0,0.0,-1.0,3.0,6.0,-2.0,-1.0,-4.0,0.0,0.0,0.0,-1.0,-3.0,0.0,-1.0,6.0,0.0,1.0,-1.0,-1.0,0.0,0.0,1.0,66.0,35.0,1.0,-1.0,0.0,0.0,-2.0,-1.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,-2.0,0.0,0.0,-1.0,-1.0,0.0,-2.0,2.0,16.0,0.0,-2.0,0.0,-1.0,-2.0,0.0,0.0,0.0,28.0,0.0,2.0,1.0,-1.0,0.0,0.0,2.0,-1.0,-3.0,2.0,1.0,1.0,3.0,-1.0,0.0,0.0,0.0,0.0,-2.0,-4.0,-1.0,-2.0,5.0,-1.0,1.0,1.0,0.0,1.0,0.0,0.0,19.0,-1.0,-1.0,-3.0,0.0,0.0,-2.0,13.0,-1.0,-1.0,1.0,-3.0,1.0,-3.0,-2.0,1.0,0.0,-1.0,5.0,0.0,0.0,-1.0,0.0,-2.0,6.0,-3.0,1.0,0.0,10.0,0.0,-1.0,0.0,1.0,0.0,3.0,24.0,0.0,0.0,61.0,0.0,1.0,-2.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,-1.0,4.0,-2.0,1.0,0.0,0.0,0.0,-2.0,1.0,2.0,3.0,35.0,-23.0,4.0,1.0,6.0,-2.0,-1.0,4.0,-2.0,2.0
2006-11-09,0.014683,0.0,-1.0,1.0,2.0,6.0,0.0,0.0,-1.0,0.0,3.0,3.0,2.0,4.0,-2.0,1.0,1.0,0.0,0.0,4.0,3.0,-1.0,-2.0,3.0,1.0,0.0,1.0,2.0,-4.0,-41.0,-2.0,0.0,5.0,1.0,0.0,0.0,1.0,-5.0,0.0,1.0,-22.0,0.0,-2.0,0.0,0.0,0.0,1.0,-1.0,-82.0,-79.0,1.0,1.0,2.0,3.0,3.0,3.0,0.0,0.0,2.0,-1.0,2.0,0.0,0.0,1.0,2.0,1.0,0.0,3.0,-3.0,1.0,2.0,-4.0,-23.0,0.0,2.0,2.0,1.0,6.0,0.0,1.0,2.0,-21.0,-1.0,-14.0,3.0,0.0,-1.0,0.0,-1.0,-3.0,-2.0,-4.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,-6.0,-5.0,0.0,0.0,0.0,0.0,0.0,0.0,-23.0,2.0,1.0,6.0,1.0,0.0,4.0,-28.0,2.0,1.0,0.0,-8.0,-7.0,4.0,-7.0,-4.0,0.0,0.0,-2.0,1.0,0.0,1.0,0.0,1.0,-5.0,-8.0,5.0,0.0,-12.0,0.0,1.0,0.0,1.0,0.0,-2.0,-27.0,0.0,0.0,-52.0,0.0,1.0,1.0,0.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,4.0,-8.0,-13.0,0.0,0.0,1.0,1.0,3.0,-1.0,3.0,-4.0,-33.0,-7.0,-6.0,3.0,-10.0,3.0,-2.0,-13.0,2.0,-8.0


,0062761Q_US_Equity_return
date,
2006-11-08,0.003296
2006-11-09,0.014683
